In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
#사용자가 추천받은 자료받아오기

click = "BerryBlossom White"

In [3]:
tea_data = pd.read_csv('teaable.csv')
rating_data = pd.read_csv('user.csv')

#삼각함수(sin 이용) 아이템기반추천알고리즘#

In [4]:
tea_data.head(2)

,차품목,효능번호,카페인,향기준,contentId,"맛,향",personId
0,Bengal Spice,1,X,2,a1,"계피, 생강",1
1,BerryBlossom White,1,O,1,a2,과일향(베리향),2


In [5]:
rating_data.head(2)

,userid,rating,contentId
0,1,10,a62
1,2,2,a34


In [10]:
user_tea_rating = pd.merge(rating_data, tea_data, on = 'contentId')

In [11]:
user_tea_rating.head(2)

,userid,rating,contentId,차품목,효능번호,카페인,향기준,"맛,향",personId
0,1,10,a62,ZEN™-green tea,3,O,3,시원한 향,62
1,21,4,a62,ZEN™-green tea,3,O,3,시원한 향,62


In [12]:
tea_user_rating = user_tea_rating.pivot_table('rating', index = '차품목',columns = 'userid').fillna(0)
user_tea_rating = user_tea_rating.pivot_table('rating',index = 'userid', columns = '차품목').fillna(0)

In [13]:
user_tea_rating.head(5)

차품목,Bengal Spice,BerryBlossom White,Black Cherry Berry,Breakfast in Paris Black Tea,ButterScotch Blondie,Candy Cane Lane,Chai Green Tea,Chamomile Nights Herbal Tea,China Green Tips,Choco,...,"Yogi Tea, 여성용 라즈베리 잎",ZEN™-green tea,요기티 Yogi Tea 라즈베리 패션 퍼펙트 에너지 티 16티백,요기티 Yogi Tea 베드타임 티 16티백,요기티 Yogi Tea 스윗 탠저린 파지티브 에너지 티 16티백,요기티 Yogi Tea 스토맥 이즈 티 16티백,요기티 Yogi Tea 진저 티 16티백,요기티 Yogi Tea 콜드 시즌 티 16티백,요기티 Yogi Tea 트롯 컴포트 티 허니 레몬 16티백,요기티 Yogi Tea 피치 디톡스 티 16티백
userid,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
tea_user_rating.head(5)

userid,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
차품목,,,,,,,,,,,,,,,,,,,,,
Bengal Spice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BerryBlossom White,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Black Cherry Berry,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Breakfast in Paris Black Tea,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ButterScotch Blondie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
#평점이 비슷하다는 것을 코사인 유사도로 측정해서 확인

item_based_collabor = cosine_similarity(tea_user_rating)
item_based_collabor

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [16]:
print(tea_user_rating.shape)
print(item_based_collabor.shape)

(65, 100)
(65, 65)


In [17]:
item_based_collabor = pd.DataFrame(data = item_based_collabor, index = tea_user_rating.index, columns = tea_user_rating.index)

In [18]:
item_based_collabor.head()

차품목,Bengal Spice,BerryBlossom White,Black Cherry Berry,Breakfast in Paris Black Tea,ButterScotch Blondie,Candy Cane Lane,Chai Green Tea,Chamomile Nights Herbal Tea,China Green Tips,Choco,...,"Yogi Tea, 여성용 라즈베리 잎",ZEN™-green tea,요기티 Yogi Tea 라즈베리 패션 퍼펙트 에너지 티 16티백,요기티 Yogi Tea 베드타임 티 16티백,요기티 Yogi Tea 스윗 탠저린 파지티브 에너지 티 16티백,요기티 Yogi Tea 스토맥 이즈 티 16티백,요기티 Yogi Tea 진저 티 16티백,요기티 Yogi Tea 콜드 시즌 티 16티백,요기티 Yogi Tea 트롯 컴포트 티 허니 레몬 16티백,요기티 Yogi Tea 피치 디톡스 티 16티백
차품목,,,,,,,,,,,,,,,,,,,,,
Bengal Spice,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.219953,0.0,0.0,0.0
BerryBlossom White,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
Black Cherry Berry,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
Breakfast in Paris Black Tea,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
ButterScotch Blondie,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [19]:
def get_item_based_collabor(title):
    return item_based_collabor[title].sort_values(ascending=False)[:6]

In [20]:
a= get_item_based_collabor(click)

In [21]:
a

차품목
BerryBlossom White      1.000000
Vanilla Bean Macaron    0.215249
Rose                    0.195180
Cinnamon Apple Spice    0.046004
Tumeric Orange          0.000000
Tumeric Chai            0.000000
Name: BerryBlossom White, dtype: float64